In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
feat_thresh_arr = [{'name':'hemoglobin', 'thresh':[12], 'std':2},
                   {'name':'ferritin', 'thresh':[30, 100], 'std':2},
                   {'name':'ret_count', 'thresh':[2], 'std':0.2},
                   {'name':'segmented_neutrophils', 'thresh':[0], 'std':2},
                   {'name':'tibc', 'thresh':[450], 'std':50},
                   {'name':'mcv', 'thresh':[80,100], 'std':2}]

anemias_features_dict = {'Vitamin B12/Folate deficiency anemia': ['mcv', 'segmented_neutrophils'], 
                         'Unspecified anemia': ['mcv', 'segmented_neutrophils'], 
                         'Anemia of chronic disease': ['mcv', 'ferritin', 'tibc'], 
                         'Iron deficiency anemia': ['mcv', 'ferritin', 'tibc'], 
                         'Hemolytic anemia': ['mcv', 'ret_count'], 
                         'Aplastic anemia': ['mcv', 'ret_count']}

#### The data

In [3]:
all_df= pd.read_csv('../../final/data/dataset_with_inconclusive.csv')
all_df = all_df.fillna(-1)
all_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.875670,161.104070,3.981397,-1.000000,358.191297,77.370085,-1.000000,6.918734,74.271409,0,-1.000000,441.651953,0.508604,86.888802,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,-1.000000,78.717345,-1.000000,3.999210,70.112738,0,-1.000000,414.333999,0.784442,37.659905,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,6.273777,87.682835,1,2.705387,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,-1.000000,3.773108,39.425050,0,-1.000000,353.517996,1.569724,26.393979,-1.000000,-1.000000,-1.000000,-1.000000,Hemolytic anemia
4,11.395680,-1.000000,-1.000000,1.320481,318.250045,102.206351,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
len(all_df)

70000

In [5]:
class_dict = constants.CLASS_DICT
#print(all_df.label.unique())
all_df['label'] = all_df['label'].replace(class_dict)
X = all_df.iloc[:, 0:-1]
y = all_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_df = pd.concat([X_train, y_train], axis=1)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,6.372841,40.066492,0,-1.000000,433.374720,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,7
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,6.124198,80.235998,1,0.584755,415.497280,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,3
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,-1.000000,31.093070,1,-1.000000,-1.000000,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,4
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,3.443422,72.829677,1,-1.000000,268.338920,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,5
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,2.956350,29.240909,1,-1.000000,319.299656,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,4


In [6]:
X_train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
42511,10.706860,-1.000000,-1.000000,2.097446,308.295738,104.212335,50.527083,5.084184,53.993391,1,1.418342,178.474897,0.410824,65.084260,118.607783,45.819372,21.282930,50.512056,1
60441,8.502071,-1.000000,0.106615,6.434651,-1.000000,96.250534,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6
18545,17.134148,450.994047,2.785983,3.580950,221.642272,95.404759,-1.000000,6.342099,85.202592,1,-1.000000,160.711704,1.004988,51.333192,-1.000000,-1.000000,-1.000000,-1.000000,0
41548,12.600706,307.881516,2.808250,2.309362,-1.000000,91.591957,68.163818,6.493743,83.937031,1,1.412507,232.325309,0.734399,8.894795,79.857627,26.676604,2.402901,73.780752,5
44111,11.708159,-1.000000,5.601042,-1.000000,-1.000000,92.100873,46.823176,5.062220,41.347210,1,2.344456,182.530896,0.849301,93.566858,54.558476,74.368283,25.629368,85.386298,5


In [7]:
# X_test_df.to_csv('../../data/more_features/train_sets/test_set_constant.csv', index=False)

In [8]:
def get_dict(dict_arr, feat_name):
    for dictionary in dict_arr:
        if dictionary['name']==feat_name:
            return dictionary

In [9]:
def feat_noisiness(df, feat_name, frac, mean, std):
    df[feat_name] = df[feat_name].sample(frac=1-frac)
    if feat_name ==  'segmented_neutrophils': #how was this a good idea
        df[feat_name] = df[feat_name].fillna(-1)
    else:
        nan_index = df[df[feat_name].isna()].index
        fill_values = np.random.normal(mean, std, size=len(nan_index))
        df[feat_name] = df[feat_name].fillna(pd.Series(fill_values, index=nan_index))
    return df[feat_name]

In [10]:
total_frac = 0.2

In [11]:
noisy_anemia_dfs_list = []
for label in X_train_df.label.unique():
    anemia = constants.ACTION_SPACE[label]
    print(f'{label} - {constants.ACTION_SPACE[label]}')
    if anemia not in ['No anemia', 'Inconclusive diagnosis']:
        anemia_df = X_train_df[X_train_df.label==label]
        feature_num = len(anemias_features_dict[anemia])
        frac = total_frac/feature_num
        for feat in anemias_features_dict[anemia]:
            feat_info = get_dict(feat_thresh_arr, feat)
            threshes, std = feat_info['thresh'], feat_info['std']
            print(f'feat:{feat}, threshes:{threshes}, std:{std}')
            for thresh in threshes:
                anemia_df[feat] = feat_noisiness(anemia_df, feat, frac/len(threshes), thresh, std)
        noisy_anemia_dfs_list.append(anemia_df)

1 - Vitamin B12/Folate deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
6 - Aplastic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
0 - No anemia
5 - Hemolytic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
2 - Unspecified anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
4 - Iron deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50
3 - Anemia of chronic disease
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50
7 - Inconclusive diagnosis


In [12]:
noisy_anemia_dfs_list[0].describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
count,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.0
mean,9.510462,114.590630,0.450322,3.072438,222.439107,101.278930,60.740629,2.655829,44.955790,0.570596,-0.328245,190.804228,0.515078,53.875010,27.921594,17.196119,4.825462,39.743601,1.0
std,2.012629,159.495280,2.190672,2.319099,170.697292,5.119053,81.393837,2.958628,30.448633,0.495026,1.199499,160.689178,1.039171,50.050921,42.471626,25.528546,9.353014,49.167522,0.0
min,6.001058,-1.000000,-1.000000,-1.000000,-1.000000,73.984735,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.0
25%,7.765217,-1.000000,-1.000000,1.228175,-1.000000,100.904462,-1.000000,-1.000000,22.535828,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.0
50%,9.513341,-1.000000,-1.000000,3.159235,227.841189,102.289878,-1.000000,3.434806,46.834711,1.000000,-1.000000,215.421829,0.755746,46.352713,-1.000000,-1.000000,-1.000000,-1.000000,1.0
75%,11.278146,232.291861,1.846724,5.034182,369.421111,103.654608,125.163190,5.225865,71.070023,1.000000,0.213008,332.652512,1.369483,97.749980,60.103302,34.952750,9.295972,84.215031,1.0
max,12.999126,499.887714,5.999175,6.999834,519.904998,106.103423,249.884526,6.999465,94.994058,1.000000,2.999847,449.981047,1.999901,149.989051,129.998850,79.988473,29.993930,139.998628,1.0


In [13]:
no_df = X_train_df[X_train_df.label==constants.CLASS_DICT['No anemia']]
inconc_df = X_train_df[X_train_df.label==constants.CLASS_DICT['Inconclusive diagnosis']]
len(no_df), len(inconc_df)

(8000, 4395)

In [14]:
noisy_anemia_dfs_list.append(no_df)
noisy_anemia_dfs_list.append(inconc_df)
len(noisy_anemia_dfs_list)

8

In [15]:
combined_df = pd.concat(noisy_anemia_dfs_list, axis=0)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,9.606630,-1.000000,1.589280,3.963014,-1.000000,86.966746,-1.000000,-1.000000,-1.000000,0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6
1,13.212575,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,31.358657,6.937041,55.368728,0,-1.0,205.368824,1.882108,60.056279,-1.000000,15.468964,-1.000000,134.444898,0
2,15.045056,372.958469,5.749105,2.533885,418.786504,100.425625,61.517834,3.961317,48.946126,1,-1.0,313.515257,1.507701,0.389689,79.814043,66.934538,22.539608,96.167367,0
3,11.352055,-1.000000,3.273160,0.927635,516.314543,77.318996,192.229667,5.680136,57.272036,0,-1.0,389.418806,1.352815,25.877380,118.834370,49.949523,19.359368,124.014291,4
4,11.766908,-1.000000,0.649533,6.192704,434.022888,102.644303,104.242624,3.608860,82.481569,1,-1.0,306.480466,1.642017,100.143806,-1.000000,6.898158,-1.000000,102.831066,1


#### The correlated features

In [16]:
def get_rbc(row):
    if row['mcv'] == -1:
        return -1
    else:
        return (30*row['hemoglobin'])/row['mcv']

def get_tsat(row):
    if (row['serum_iron']==-1) | (row['tibc']==-1):
        return -1
    else:
        return (row['serum_iron']/row['tibc'])*100

In [17]:
combined_df['rbc'] = combined_df.apply(lambda row: get_rbc(row), axis=1)
combined_df['tsat'] = combined_df.apply(lambda row: get_tsat(row), axis=1)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label,tsat
0,9.606630,-1.000000,1.589280,3.963014,-1.000000,86.966746,-1.000000,3.313898,-1.000000,0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6,-1.000000
1,13.212575,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,31.358657,-1.000000,55.368728,0,-1.0,205.368824,1.882108,60.056279,-1.000000,15.468964,-1.000000,134.444898,0,-1.000000
2,15.045056,372.958469,5.749105,2.533885,418.786504,100.425625,61.517834,4.494388,48.946126,1,-1.0,313.515257,1.507701,0.389689,79.814043,66.934538,22.539608,96.167367,0,14.689545
3,11.352055,-1.000000,3.273160,0.927635,516.314543,77.318996,192.229667,4.404631,57.272036,0,-1.0,389.418806,1.352815,25.877380,118.834370,49.949523,19.359368,124.014291,4,37.231116
4,11.766908,-1.000000,0.649533,6.192704,434.022888,102.644303,104.242624,3.439131,82.481569,1,-1.0,306.480466,1.642017,100.143806,-1.000000,6.898158,-1.000000,102.831066,1,24.017771


#### Randomly modifying to no anemia

In [18]:
anemic_df = combined_df[combined_df.label != constants.CLASS_DICT['No anemia']]
non_anemic_df = combined_df[combined_df.label == constants.CLASS_DICT['No anemia']]
len(anemic_df), len(non_anemic_df)

(48000, 8000)

In [19]:
anemic_df.label.value_counts()

3    7527
4    7261
6    7223
5    7220
1    7203
2    7171
7    4395
Name: label, dtype: int64

In [20]:
anemic_df['label'] = anemic_df['label'].sample(frac=1-0.1)
anemic_df['label'] = anemic_df['label'].fillna(constants.CLASS_DICT['No anemia'])
anemic_df.label.value_counts()

3.0    6800
5.0    6543
4.0    6516
1.0    6504
6.0    6461
2.0    6425
0.0    4800
7.0    3951
Name: label, dtype: int64

#### Finalizing 

In [21]:
final_df = pd.concat([non_anemic_df, anemic_df], axis=0)
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label,tsat
0,10.490380,-1.0,0.113630,4.738495,-1.000000,99.895590,-1.000000,3.150403,46.021619,0,-1.000000,179.716334,0.730432,24.332066,-1.000000,-1.000000,-1.000000,-1.000000,6.0,-1.0
1,10.309792,-1.0,2.067524,0.404102,-1.000000,99.015814,208.280364,3.123680,71.203174,1,1.470031,180.751558,1.665866,13.065579,115.010315,60.042365,4.992607,88.383056,0.0,-1.0
2,8.374113,-1.0,-1.000000,3.453598,448.296575,103.808207,-1.000000,2.420072,51.081069,1,-1.000000,-1.000000,1.766657,132.632967,-1.000000,-1.000000,-1.000000,-1.000000,1.0,-1.0
3,10.829884,-1.0,2.530899,-1.000000,-1.000000,90.664501,185.317623,3.583503,37.678416,1,-1.000000,220.475548,0.345588,31.763527,-1.000000,68.928513,-1.000000,100.232195,5.0,-1.0
4,6.650546,-1.0,-1.000000,0.528332,467.058906,101.393019,-1.000000,1.967753,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.0,-1.0


In [22]:
final_df.label.value_counts()

0.0    12800
3.0     6800
5.0     6543
4.0     6516
1.0     6504
6.0     6461
2.0     6425
7.0     3951
Name: label, dtype: int64

In [ ]:
final_df.to_csv('../../final/train_set_noisy_2.csv', index=False)

In [23]:
utils.get_dt_performance(final_df, 'numeric')

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
len(final_df)